# Advent of Code 2024 Day 9 

### Setup

In [1]:
from aocd import get_data, submit

day = 9
year = 2024


In [2]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

'2333133121414131402'

In [3]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

'432111411641322054157057763724522244402166159217126386588522473227382586931248316770856781767410884015472481402048831959743351815379489141971184317670536745767945237644221787373392627291744595596588869346866297675523326156492121268985867082266092104552168932594386159479249021712032716979272323749958721690459659987212171685926395601099473345781359611948721939233573637224543716984589864318987224509824659376852571938082385869298284199727243315582615455758924144937074645269474446112080325781753587117286125839954079161467106766969048567557593377974863994861293797415844336568138133752677677455774170967639882828631693665852722264479071208165207924371671861171469959126488964090829687763339716764545182407069254627179061261736849238791481995745164512697765213699326454362398438427336299873758642884422842729663376981586380566547396781555198149525937743287578319011347750908163471088781085343771332080296890632225891429707036985357312585947560518336316666532946396551686216129792888861727526515510461

##### Data Parsing

In [4]:
def parse_data(raw_data:str):
    return [int(x) for x in raw_data] 

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

[2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2]

### Part One!

In [5]:
use_sample_data = False
part = 'a'

In [6]:
data = sample_data if use_sample_data else test_data

str(data)

'[4, 3, 2, 1, 1, 1, 4, 1, 1, 6, 4, 1, 3, 2, 2, 0, 5, 4, 1, 5, 7, 0, 5, 7, 7, 6, 3, 7, 2, 4, 5, 2, 2, 2, 4, 4, 4, 0, 2, 1, 6, 6, 1, 5, 9, 2, 1, 7, 1, 2, 6, 3, 8, 6, 5, 8, 8, 5, 2, 2, 4, 7, 3, 2, 2, 7, 3, 8, 2, 5, 8, 6, 9, 3, 1, 2, 4, 8, 3, 1, 6, 7, 7, 0, 8, 5, 6, 7, 8, 1, 7, 6, 7, 4, 1, 0, 8, 8, 4, 0, 1, 5, 4, 7, 2, 4, 8, 1, 4, 0, 2, 0, 4, 8, 8, 3, 1, 9, 5, 9, 7, 4, 3, 3, 5, 1, 8, 1, 5, 3, 7, 9, 4, 8, 9, 1, 4, 1, 9, 7, 1, 1, 8, 4, 3, 1, 7, 6, 7, 0, 5, 3, 6, 7, 4, 5, 7, 6, 7, 9, 4, 5, 2, 3, 7, 6, 4, 4, 2, 2, 1, 7, 8, 7, 3, 7, 3, 3, 9, 2, 6, 2, 7, 2, 9, 1, 7, 4, 4, 5, 9, 5, 5, 9, 6, 5, 8, 8, 8, 6, 9, 3, 4, 6, 8, 6, 6, 2, 9, 7, 6, 7, 5, 5, 2, 3, 3, 2, 6, 1, 5, 6, 4, 9, 2, 1, 2, 1, 2, 6, 8, 9, 8, 5, 8, 6, 7, 0, 8, 2, 2, 6, 6, 0, 9, 2, 1, 0, 4, 5, 5, 2, 1, 6, 8, 9, 3, 2, 5, 9, 4, 3, 8, 6, 1, 5, 9, 4, 7, 9, 2, 4, 9, 0, 2, 1, 7, 1, 2, 0, 3, 2, 7, 1, 6, 9, 7, 9, 2, 7, 2, 3, 2, 3, 7, 4, 9, 9, 5, 8, 7, 2, 1, 6, 9, 0, 4, 5, 9, 6, 5, 9, 9, 8, 7, 2, 1, 2, 1, 7, 1, 6, 8, 5, 9, 2, 6, 3, 9, 5, 6, 0, 1,

In [7]:
from typing import Dict, List, Union, Literal

MemoryBlock = Union[int, Literal['.']]
Memory = List[MemoryBlock]

FileInfo = Dict[int, Dict[str, int]]
FreeSpaceInfo = Dict[int, List[int]]

StorageInfo = Dict[Union[Literal['freespace', 'files', 'max_file_size', 'max_freespace_size']], Union[Dict[int, FileInfo], FreeSpaceInfo]]


def is_freespace(block:MemoryBlock):
    return block == '.'

def is_filespace(block:MemoryBlock):
    return not is_freespace(block)


In [8]:
def load_memory(discrete_representation: list[int]) -> Memory:
    memory = []

    for i in range(len(discrete_representation)):
        load_file = i % 2 == 0
        file_idx = i // 2
        block = file_idx if load_file else '.'
        memory += [block] * discrete_representation[i]
    
    return memory

load_memory(data)[:10]

[0, 0, 0, 0, '.', '.', '.', 1, 1, '.']

In [9]:
import heapq

def load_storage_info(discrete_representation: list[int]) -> StorageInfo:
    storage_info: StorageInfo = {'files': {}, 'freespace': {}, 'max_file_size': 0, 'max_freespace_size': 0}
    offset = 0

    for i, block_size in enumerate(discrete_representation):
        load_file = i % 2 == 0
        idx = i // 2
        
        if load_file:
            storage_info['files'][idx] = {
                'file_location': offset,
                'file_size': block_size
            }

            if block_size > storage_info['max_file_size']:
                storage_info['max_file_size'] = block_size
        
        else:
            storage_info['freespace'].setdefault(block_size, [])
            heapq.heappush(storage_info['freespace'][block_size], offset)
            
            if block_size > storage_info['max_freespace_size']:
                storage_info['max_freespace_size'] = block_size

        offset += block_size
    
    return storage_info

In [10]:
def optimize_memory_with_file_fragmentation(memory: list[MemoryBlock], storage_info: dict[int: int]) ->  Memory:
    optimized_memory = memory[:]

    i, j = 0, len(memory) - 1

    while i < j:
        frontblock, endblock = optimized_memory[i], optimized_memory[j]

        if is_filespace(frontblock):
            i += 1
            continue

        elif is_freespace(endblock):
            j -= 1
            continue

        optimized_memory[i] = optimized_memory[j]
        optimized_memory[j] = '.'

        i += 1
        j -= 1

    return optimized_memory

In [11]:
def optimize_memory_without_file_fragmentation(memory: list[MemoryBlock], storage_info: StorageInfo) -> Memory:
    optimized_memory = memory[:]
    get_candidate_freespace_sizes = lambda size: ( k for k in storage_info['freespace'] if k >= size and len(storage_info['freespace'][k]) > 0 )

    for file, file_info in reversed(storage_info['files'].items()):
        file_size = file_info['file_size']
        file_location = file_info['file_location']

        for freespace_size in sorted(get_candidate_freespace_sizes(file_size), key=lambda k: storage_info['freespace'][k][0]):
            if freespace_size < file_size:
                continue
            
            if (storage_info['freespace'][freespace_size][0] >= file_location):
                continue

            idx = heapq.heappop(storage_info['freespace'][freespace_size])
            for offset in range(file_size):
                optimized_memory[idx + offset] = file
                optimized_memory[file_location + offset] = '.'
            
            heapq.heappush(storage_info['freespace'][file_size], file_location)
            
            if file_size != freespace_size:
                leftover_freespace = freespace_size - file_size
                leftover_idx = idx + file_size
                storage_info['freespace'].setdefault(leftover_freespace, [])
                heapq.heappush(storage_info['freespace'][leftover_freespace], leftover_idx)
            
            break
        
    return optimized_memory

In [12]:
def optimize_memory(memory: list[MemoryBlock], storage_info: dict[int: int], file_fragmentation=True) -> Memory:
    if file_fragmentation:
        return optimize_memory_with_file_fragmentation(memory=memory, storage_info=storage_info)
        
    else:
        return optimize_memory_without_file_fragmentation(memory=memory, storage_info=storage_info)


In [13]:
def generate_checksum(memory: list[MemoryBlock]) -> int:
    checksum = 0
    for i, block in enumerate(memory):
        checksum += block * i if is_filespace(block) else 0
    
    return checksum

In [14]:
storage_info = load_storage_info(data)

memory = load_memory(data)

optimized_memory = optimize_memory(memory, storage_info)

checksum = generate_checksum(optimized_memory)

part_a_answer = checksum
part_a_answer

6340197768906

In [15]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

aocd will not submit that answer again. At 2024-12-09 11:13:56.419986-05:00 you've previously submitted 6340197768906 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


### Part Two!

In [16]:
use_sample_data = False
part='b'

In [17]:
data = sample_data if use_sample_data else test_data

str(data[:100]), len(data)

('[4, 3, 2, 1, 1, 1, 4, 1, 1, 6, 4, 1, 3, 2, 2, 0, 5, 4, 1, 5, 7, 0, 5, 7, 7, 6, 3, 7, 2, 4, 5, 2, 2, 2, 4, 4, 4, 0, 2, 1, 6, 6, 1, 5, 9, 2, 1, 7, 1, 2, 6, 3, 8, 6, 5, 8, 8, 5, 2, 2, 4, 7, 3, 2, 2, 7, 3, 8, 2, 5, 8, 6, 9, 3, 1, 2, 4, 8, 3, 1, 6, 7, 7, 0, 8, 5, 6, 7, 8, 1, 7, 6, 7, 4, 1, 0, 8, 8, 4, 0]',
 19999)

In [18]:
storage_info = load_storage_info(data)

memory = load_memory(data)

optimized_memory = optimize_memory(memory, storage_info, file_fragmentation=False)

checksum = generate_checksum(optimized_memory)

part_b_answer = checksum
part_b_answer

6363913128533

In [19]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)

aocd will not submit that answer again. At 2024-12-09 17:30:26.893241-05:00 you've previously submitted 6363913128533 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Optimization Results 

The initial approach for Part 2's optimization without file fragmentation was let's say, less that optimal.... On the test data it was able to correctly optimize the memory but runs for a whopping 1.5 minutes. As such I spent some time optimizing the solution to make use of a tabulated set of freespace positions. Each size of freespace (0 - 9) has an associated heap. By compaing the top of the heap (peeking) of the freespace sizes that would fit the file we are able to quickly retrieve the location that the file is supposed to move to. This removes a linear scan of all the memory for each file and cuts down the runtime drastically. 

This section showcases the two times using %timeit for comparison 

The original un-optimal implementation. (it is clean tho)

In [20]:
def optimize_memory_without_file_fragmentation_original(memory: list[MemoryBlock], storage_info: dict[int: int]) -> list[MemoryBlock]:
    optimized_memory = memory[:]

    for file, file_info in reversed(storage_info['files'].items()):
        file_size = file_info['file_size']
        j = file_info['file_location']

        for i in range(len(optimized_memory)):
            if i >= j:
                break

            memory_slice = optimized_memory[i:i + file_size]
            is_freespace_available = all([ is_freespace(x) for x in memory_slice])

            if not is_freespace_available:
                continue

            for offset in range(file_size):
                optimized_memory[i + offset] = file
                optimized_memory[j + offset] = '.'
            
            break
    
    return optimized_memory

First the un-optimzied approach 

In [ ]:
%timeit optimize_memory_without_file_fragmentation_original(load_memory(data), load_storage_info(data))

59.5 s ± 286 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now the Optimized approach

In [24]:
%timeit optimize_memory_without_file_fragmentation(load_memory(data), load_storage_info(data))

18.9 ms ± 150 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
